In [1]:
import pandas as pd
import numpy as np
import os
import sys
import plotly
import plotly.graph_objs as go
    
%load_ext autoreload
%autoreload 1

pd.set_option("display.max_columns",201)
pd.set_option("display.max_colwidth",101)
pd.set_option("display.max_rows",500)

In [2]:
from arctic import Arctic, CHUNK_STORE

conn = Arctic('10.213.120.5')
conn.initialize_library('entsoe', lib_type=CHUNK_STORE)
conn.list_libraries()
lib = conn['entsoe']

c:\users\a0743104\appdata\local\programs\python\python38-32\lib\site-packages\arctic\_util.py:6: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
c:\users\a0743104\appdata\local\programs\python\python38-32\lib\site-packages\arctic\store\_pandas_ndarray_store.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import DataFrame, Series, Panel
Library created, but couldn't enable sharding: no such command: 'enablesharding'. This is OK if you're not 'admin'


In [3]:
# Input country

list_countries = ['DE','FR','ES','BE','PL']

In [4]:
# function to change timezone from UTC to local time

def changing_timezone(x):
    ts = x.index.tz_localize('utc').tz_convert('Europe/Brussels')
    y = x.set_index(ts)
    return y.tz_localize(None)

In [44]:
def create_plot_monthly(df=None,
                country=None,
                list_countries=None):
    
    from datetime import datetime
    from plotly.subplots import make_subplots
    from datetime import timedelta
    
    #-----------------------------------------------------------------------------
    l  = list_countries.index(country)
    if ((l+1) % 2) == 0:
            k = 2
    else:
            k = 1    

    trace = go.Scatter(x=df.index, 
               y=df['cap_price_Solar'],
               name = 'Solar PV',
               line_color='yellow',
               showlegend=False if l>0 else True,
               legendgroup='g1',
               hovertemplate='%{x},%{y:.1f}',
               #visible = 'legendonly' if i<x.year else True
                      )
    fig.append_trace(trace, -(-(l+1)//2), k)
    #-----------------------------------------------------------------------------
    trace = go.Scatter(x=df.index, 
               y=df['cap_price_Wind Onshore'],
               name = 'Onshore Wind',
               line_color='royalblue',
               showlegend=False if l>0 else True,
               legendgroup='g2',
               hovertemplate='%{x},%{y:.1f}',
               #visible = 'legendonly' if i<x.year else True
                         )
    fig.append_trace(trace, -(-(l+1)//2), k)
    #-----------------------------------------------------------------------------
    try:
        trace = go.Scatter(x=df.index, 
               y=df['cap_price_Wind Offshore'],
               name = 'Offshore Wind',
               line_color='green',
               showlegend=False if l>0 else True,
               legendgroup='g3',
               hovertemplate='%{x},%{y:.1f}',
               #visible = 'legendonly' if i<x.year else True
            )
        fig.append_trace(trace, -(-(l+1)//2), k) 
    except KeyError:
        pass
    #-----------------------------------------------------------------------------          
    trace = go.Scatter(
               x = df.index, 
               y = df['DayAheadPrices'+'_'+country], 
               name = 'Baseload',
               line = dict(color='indianred'),
               showlegend= False if l>0 else True,
               legendgroup='g4',
               hovertemplate='%{x},%{y:.1f}',
               #visible = 'legendonly' if i<x.year else True
           )
    fig.add_trace(trace, -(-(l+1)//2), k)
 #-----------------------------------------------------------------------------
    return fig

In [5]:
def create_plot(df=None,
                country=None,
                list_countries=None):
    
    from datetime import datetime
    from plotly.subplots import make_subplots
            
    #-----------------------------------------------------------------------------
    l  = list_countries.index(country)
    if ((l+1) % 2) == 0:
            k = 2
    else:
            k = 1
            
    trace = go.Scatter(x=df.index, 
            y=df['cap_price_Solar'],
            name = 'Solar PV',
            line_color='yellow',
            showlegend=False if l>0 else True,
            legendgroup='g1',
            hovertemplate='%{x},%{y:.1f}')
    fig.append_trace(trace, -(-(l+1)//2), k)
    #-----------------------------------------------------------------------------
    trace = go.Scatter(x=df.index, 
            y=df['cap_price_Wind Onshore'],
            name = 'Onshore Wind',
            line_color='royalblue',
            showlegend=False if l>0 else True,
            legendgroup='g2',
            hovertemplate='%{x},%{y:.1f}')
    fig.append_trace(trace, -(-(l+1)//2), k)
     #-----------------------------------------------------------------------------
    try:
        trace = go.Scatter(x=df.index, 
            y=df['cap_price_Wind Offshore'],
            name = 'Offshore Wind',
            line_color='green',
            showlegend=False if l>0 else True,
            legendgroup='g3',
            hovertemplate='%{x},%{y:.1f}')
        fig.append_trace(trace, -(-(l+1)//2), k) 
    except KeyError:
        pass
   #-----------------------------------------------------------------------------          
    trace = go.Scatter(
            x = df.index, 
            y = df['DayAheadPrices'+'_'+country], 
            name = 'Baseload',
            line = dict(color='indianred'),
            showlegend= False if l>0 else True,
            legendgroup='g4',
            hovertemplate='%{x},%{y:.1f}'
        )
    fig.add_trace(trace, -(-(l+1)//2), k)
 #-----------------------------------------------------------------------------
    return fig

In [6]:
# define dates

from datetime import datetime
from datetime import timedelta
from datetime import date

hist_dir = 'hist_data/'
hist_filename = 'df_hist_cap_price_DE.p'

df_hist = pd.read_pickle(os.path.join(hist_dir, hist_filename))
start_date = df_hist.index[-1] + timedelta(days = - 1)

#ref_date = datetime(year=2016, month=1, day=1).date()
#start_date = ref_date + timedelta(days = - 1)

end_date = date.today().replace(day=1)

In [45]:
import calendar 
import xlsxwriter
  
writer_1 = pd.ExcelWriter('hist_cap_price_monthly.xlsx', engine='xlsxwriter')
writer_2 = pd.ExcelWriter('hist_cap_price_yearly.xlsx', engine='xlsxwriter')
workbook_1 = writer_1.book
workbook_2 = writer_2.book

from datetime import datetime
from plotly.subplots import make_subplots
    
    #-----------------------------------------------------------------------------
fig = plotly.subplots.make_subplots(
            rows=3, cols=2, 
            subplot_titles = list_countries,
            shared_xaxes=False,
            vertical_spacing=0.1)

for i in list_countries:
    
    pickle_filename = 'df_hist_cap_price_'+i+'.p'
    df_hist_cap_price = pd.read_pickle(os.path.join(hist_dir,pickle_filename))

    # Read Spot price
    var = 'DayAheadPrices_12.1.D'

    prefix = var + '_' + i 
    
    df_DA_price = lib.read(prefix, chunk_range=pd.date_range(start_date, end_date))
    
    # Read power generation data
    var = 'AggregatedGenerationPerType_16.1.B_C'
    prefix = var + '_' + i 
    
    df_gen = lib.read(prefix,chunk_range=pd.date_range(start_date, end_date))
    
    # convert 15 min data to hourly data
    df_gen = df_gen.resample('H').mean()
    
    # changing timezones 
    df_gen = changing_timezone(df_gen)
    
    # Extract Solar and Wind data
    
    var = 'ActualGenerationOutput'
    res_tech = ['Solar','Wind Onshore', 'Wind Offshore']
    
    df_res_gen = pd.DataFrame()
    prefix = var + ' ' + i 
    for j in res_tech:
        try:
            df_res_gen = pd.concat([df_res_gen,df_gen[prefix+' '+j]],axis=1)
        except KeyError:
            pass 
        
    df_res_gen.index = pd.to_datetime(df_res_gen.index)
    
    df_DA_price =df_DA_price[~df_DA_price.index.duplicated()]
    df_res_gen =df_res_gen[~df_res_gen.index.duplicated()]
    
   # merging data to a single dataframe

    var = [df_DA_price,df_res_gen]     
    df_merge = pd.DataFrame(columns=[])
    
    for j in var:
        df_merge = pd.merge(df_merge, j,how='outer',right_index=True, left_index=True)
    
    for j in res_tech:
        try:
            df_merge['t_cap'+' '+j] = df_merge['DayAheadPrices_'+i]*df_merge[prefix+' '+j]
        except KeyError:
            pass
    
    #df_h= df_merge.iloc[(df_merge.index.year >=2016)&(df_merge.index.date<end_date)]
    
    df_merge = df_merge.iloc[(df_merge.index.date<end_date)]
    
    df_h = pd.concat([df_hist_cap_price,df_merge])
    
    df_h = df_h[~df_h.index.duplicated(keep='last')]
    
    #df_d = df_h.groupby(df_h.index.date).mean()
    
    df_m = df_h.groupby([(df_h.index.year),(df_h.index.month)]).mean()
    
    df_y = df_h.groupby([(df_h.index.year)]).mean()
    
    for j in res_tech:
        try:
            df_m['cap_price' +'_'+j] = df_m['t_cap' +' '+ j]/df_m[prefix + ' ' + j]
            df_y['cap_price' +'_'+j] = df_y['t_cap' +' '+ j]/df_y[prefix + ' ' + j]
        except KeyError:
            pass
        
    df_m['qf_Solar'] = df_m['cap_price_Solar']/df_m['DayAheadPrices_'+i]
    df_m['qf_Wind Onshore'] = df_m['cap_price_Wind Onshore']/df_m['DayAheadPrices_'+i]
    
    df_y['qf_Solar'] = df_y['cap_price_Solar']/df_y['DayAheadPrices_'+i]
    df_y['qf_Wind Onshore'] = df_y['cap_price_Wind Onshore']/df_y['DayAheadPrices_'+i]
    try:
        df_m['qf_Wind Offshore'] = df_m['cap_price_Wind Offshore']/df_m['DayAheadPrices_'+i]
        df_y['qf_Wind Offshore'] = df_y['cap_price_Wind Offshore']/df_y['DayAheadPrices_'+i]
    except KeyError:
        pass
        
    df = df_m
    
    #Use calendar library for abbreviations and order
    dd=dict((enumerate(calendar.month_abbr)))
    
    #rename level zero of multiindex
    df = df.rename(index=dd,level=1)
    
    #Create calendar month data type with order for sorting
    cal_dtype = pd.CategoricalDtype(list(calendar.month_abbr), ordered=True)
    
    #Change the dtype of the level zero index
    df.index = df.index.set_levels(df.index.levels[1].astype(cal_dtype), level=1)
    
    try:
        df_m_final = df[['DayAheadPrices_'+i,'cap_price_Solar','qf_Solar', 
                       'cap_price_Wind Onshore','qf_Wind Onshore',
                       'cap_price_Wind Offshore', 'qf_Wind Offshore']]
        df_y_final = df_y[['DayAheadPrices_'+i,'cap_price_Solar','qf_Solar', 
                       'cap_price_Wind Onshore','qf_Wind Onshore',
                       'cap_price_Wind Offshore', 'qf_Wind Offshore']]
    except KeyError:
        df_m_final = df[['DayAheadPrices_'+i,'cap_price_Solar','qf_Solar', 
                       'cap_price_Wind Onshore','qf_Wind Onshore']]
        df_y_final = df_y[['DayAheadPrices_'+i,'cap_price_Solar','qf_Solar', 
                       'cap_price_Wind Onshore','qf_Wind Onshore']]
        
    #df_m_final = df_m_final.transpose()
    #df_y_final = df_y_final.transpose()

    df_m_final.to_excel(writer_1, sheet_name=i)
    df_y_final.to_excel(writer_2, sheet_name=i)
    
    x = df_m.tail(12)
    y = x.index.get_level_values(0)
    z = x.index.get_level_values(1)

    x['date'] = pd.to_datetime(y * 10000 + z * 100 + 1, format="%Y%m%d")
    x.set_index('date', append=True, inplace=True)
    x = x.reset_index(level=[0,1])
        
    outdir = 'plots/'
 
    #fig = create_plot(df = df_y,
    #                  country = i,
    #                  list_countries = list_countries)
    
    fig = create_plot_monthly(df = x,
                    country = i,
                    list_countries = list_countries)
    
    pickle_filename = 'df_hist_cap_price_'+i+'.p'
    
    df_h.to_pickle(os.path.join(hist_dir,pickle_filename))
    
writer_1.save()
workbook_1.close()
writer_2.save()
workbook_2.close()

# Add figure title
fig.update_layout(
        title_text="Capture Price")

# Set axes titles
fig.update_yaxes(title_text="€/MWh", title_font = dict(size = 12))
        
outfile = 'hist_cap_price_monthly'+'.html'

plotly.offline.plot(fig, filename = os.path.join(outdir, outfile))

<ipython-input-45-9071ab8a35f1>:144: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-45-9071ab8a35f1>:144: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-45-9071ab8a35f1>:144: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-45-9

'plots/hist_cap_price_monthly.html'

<ipython-input-37-ce7ffd6e96d6>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [38]:
x

,level_0,level_1,DayAheadPrices_PL,ActualGenerationOutput PL Solar,ActualGenerationOutput PL Wind Onshore,t_cap Solar,t_cap Wind Onshore,cap_price_Solar,cap_price_Wind Onshore,qf_Solar,qf_Wind Onshore
date,,,,,,,,,,,
2020-08-01,2020,8,52.285565,379.905484,875.916250,21720.850230,44497.903229,57.174353,50.801550,1.093502,0.971617
2020-09-01,2020,9,53.739083,331.344306,1225.267778,18733.773584,63518.026481,56.538692,51.840118,1.052096,0.964663
2020-10-01,2020,10,52.189355,214.227177,1740.986935,12027.281426,86056.349886,56.142650,49.429636,1.075749,0.947121
2020-11-01,2020,11,53.550778,172.621889,1839.464083,10149.956921,92278.521008,58.798783,50.165981,1.098001,0.936793
2020-12-01,2020,12,56.653011,133.878710,2403.687446,8539.264323,125357.701895,63.783587,52.152247,1.125864,0.920556
2021-01-01,2021,1,54.787137,92.505922,1739.858533,5574.880934,95460.132657,60.265125,54.866606,1.099987,1.001451
2021-02-01,2021,2,58.889405,208.265514,1926.173189,12923.308741,107289.878103,62.052082,55.701055,1.053705,0.945859
2021-03-01,2021,3,59.632702,408.214737,1844.753252,25539.233866,99786.170130,62.563233,54.091879,1.049143,0.907084
2021-04-01,2021,4,59.356764,620.134916,1826.535989,36614.262235,103498.664385,59.042414,56.663906,0.994704,0.954633
